# noise model

In [1]:
import sys
sys.path.insert(0, "../source/")

from FOM_advectiondiffusion import FOM_advectiondiffusion
from FOM_advectiondiffusion_steadystate import FOM_advectiondiffusion_steadystate
from Drone import Drone
from Parameter import Parameter
from Brain import Brain
from NoiseModel import NoiseModel
from Prior import Prior

import dolfin as dl
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.linalg as la
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import math
import scipy.stats as stats

In [2]:
tStart_notebook = time.time()

## User settings

In [3]:
meshDim = 50
# meshDim=50 is probably too coarse but for writing the code it should suffice, I'm impatient

## noise model
First we test that the sampled noise is indeed in the same order of magnitude regardless of time discretization.

In [4]:
grid_t = np.linspace(0, 1, 101)
noise_model = NoiseModel(grid_t=grid_t, c_scaling=1e+3, c_diffusion = 0.01)

In [5]:
# test = noise_model.sample(n_samples = 10)

# fig, ax = plt.subplots(1,1)
# for i in range(10):
#     ax.plot(grid_t, test[:, i])
    
# ax.set_title("samples of the noise model")
# ax.set_xlabel("time")

In [6]:
# grid_t = np.linspace(0, 1, 501)
# noise_model = NoiseModel(grid_t=grid_t, c_scaling=1e+3, c_diffusion = 0.01)

# test = noise_model.sample(n_samples = 10)

# fig, ax = plt.subplots(1,1)
# for i in range(10):
#     ax.plot(grid_t, test[:, i])
    
# ax.set_title("samples of the noise model")
# ax.set_xlabel("time")

Since the noise remains in the same order of magnitude we conclude that the issue is indeed not with the way we sample but with how we evaluate the noise covariance matrix. 

## noise covariance matrix

In [7]:
grid_t_fine = np.linspace(0, 1, 2001)
noise_model_fine = NoiseModel(grid_t=grid_t_fine, c_scaling=1e+1, c_diffusion = 0.01)

sample_fine = noise_model_fine.sample(n_samples = 1)

AttributeError: 'NoiseModel' object has no attribute 'mass_matrix_sqrt'

In [ ]:
slicer = 10
grid_t_coarse = grid_t_fine[::slicer]
noise_model_coarse = NoiseModel(grid_t=grid_t_coarse, c_scaling=1e+1, c_diffusion = 0.01)
sample_coarse = sample_fine[::slicer]

In [ ]:
plt.plot(grid_t_fine, sample_fine, label = "fine dt")
plt.plot(grid_t_coarse, sample_coarse, label = "coarse dt")
plt.legend()

In [ ]:
sample_fine.T @ noise_model_fine.apply_covar_inv(d=sample_fine)

In [ ]:
sample_coarse.T @ noise_model_coarse.apply_covar_inv(d=sample_coarse)

In [ ]:
slicer = 10
grid_t_coarse = grid_t_fine[::slicer]
noise_model_coarse = NoiseModel(grid_t=grid_t_coarse, c_scaling=1e+1, c_diffusion = 0.01)
sample_coarse = sample_fine[::slicer]

In [ ]:
dt = grid_t_coarse[1]-grid_t_coarse[0]

In [ ]:
M = noise_model_coarse.mass_matrix.toarray()

In [ ]:
self = noise_model_coarse
LHS = (self.c_diffusion * self.diffusion_matrix + self.mass_matrix)

In [ ]:
v = LHS @ (grid_t_coarse**2)

In [ ]:
v.T @ sla.spsolve(M, v)

In [ ]:
4*dt/6

In [ ]:
v

In [ ]:
grid_t_coarse**2

In [ ]:
grid_t_coarse

In [ ]:
plt.plot(grid_t_coarse, grid_t_coarse**2)

In [ ]:
grid_t_coarse = np.linspace(0, 1, 1001)
noise_model_coarse = NoiseModel(grid_t=grid_t_coarse, c_scaling=1, c_diffusion = 1)

In [ ]:
v = noise_model_coarse.diffusion_matrix @ ((grid_t_coarse-1)**2*grid_t_coarse**2)

In [ ]:
sum(np.abs(v))

In [ ]:
self = noise_model_coarse
LHS = (self.c_diffusion * self.diffusion_matrix + self.mass_matrix)

In [ ]:
v = LHS @ ((grid_t_coarse-1)**2*grid_t_coarse**2)

In [ ]:
v.T @ sla.spsolve(noise_model_coarse.mass_matrix, v)

In [ ]:
yolo = sla.spsolve(noise_model_coarse.mass_matrix, v)

In [ ]:
yolo.T @ noise_model_coarse.mass_matrix @ yolo

In [ ]:
w = (grid_t_coarse-1)**2*grid_t_coarse**2

In [ ]:
plt.plot(grid_t_coarse, w)

In [ ]:
Dw = (self.diffusion_matrix @ w)
Mw = self.mass_matrix @ w

In [ ]:
Dw.T @ sla.spsolve(self.mass_matrix, Dw)

In [ ]:
Mw.T @ w

In [ ]:
Dw.T @ w

In [ ]:
Dw.T @ sla.spsolve(self.mass_matrix, Dw) + Mw.T @ w + 2*Dw.T @ w

In [ ]:
(Dw + Mw).T @ sla.spsolve(self.mass_matrix, Dw + Mw)

In [ ]:
Dw

In [ ]:
w

In [ ]:
self.diffusion_matrix.toarray()